In [ ]:
!pip install tensorflow_text
!pip install sentence_transformers
!pip install profanity-check
!pip install transformers
!pip install better_profanity

In [2]:
from better_profanity import profanity
profanity.censor("fuck off")

'****'

In [ ]:
!pip install flask_cors

In [ ]:
!pip install flask_ngrok
!pip install pyngrok==4.1.1

!ngrok authtoken 2PUA67uueLD6KUjmBkEW2XjZZSe_2rt4FLAvfahfaZrJH8iCc

In [5]:
from flask_ngrok import run_with_ngrok
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
from torch.optim import Adam
from tensorflow import keras
import tensorflow_text as text
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS, cross_origin
import urllib
import json
import logging
import tensorflow as tf
import tensorflow_text as text
import numpy as np
from tensorflow import keras
from sentence_transformers import SentenceTransformer
import csv
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import os

In [6]:
device = 'cpu'
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_tokens(["<reply>:"])
tokenizer.add_tokens(["<sentiment>:"])
tokenizer.add_tokens(["<ner>:"])
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
model_gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
model_gpt2.resize_token_embeddings(len(tokenizer))
model_gpt2 = model_gpt2.to(device)

In [7]:
with open('/content/drive/MyDrive/NLP-Project/sentence_embeddings.pkl', 'rb') as f:
    sentence_embeddings = pickle.load(f)

with open('/content/drive/MyDrive/NLP-Project/df_all.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # skip the header row
    qa_data = [{"question": row[1], "answer": row[2]} for row in reader]

In [8]:
model_sim = SentenceTransformer('bert-base-nli-mean-tokens')

In [9]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [10]:
def ret_roberta(query, res_gpt, last_line):
  q_embed = model_sim.encode(query)
  results = cosine_similarity(
      [q_embed],
      sentence_embeddings
  )
  res_gpt = res_gpt.split("<reply>:")
  res_gpt = res_gpt[1]
  print("GPT reply:", res_gpt)
  
  
  if "Reply: " in last_line:
    last_line = last_line.split("Reply: ")
    last_line = last_line[1]
  
  # largest
  larg = np.argpartition(results[0], -1)[-1]

  #second largest
  sec_lar = np.argpartition(results[0], -2)[-2]
  th_lar = np.argpartition(results[0], -2)[-3]

  combined_context = qa_data[larg]['question'] + '. ' + qa_data[sec_lar]['question']+'. '+qa_data[larg]['answer'] + '. ' + qa_data[sec_lar]['answer'] +'. ' + qa_data[th_lar]['question']+'. ' + qa_data[th_lar]['answer']
  print("context: ", query, last_line, res_gpt, combined_context)
  QA_input = {
      'question': query,
      'context': query + res_gpt + combined_context
  }
  link = ""
  res = nlp(QA_input)
  for word in combined_context.split(" "):
    if "https" in word:
        link = " link: " + word
  return res["answer"] + link

In [ ]:
ret_roberta("why do you think apple is cool"," I'll be quick to anger if I don't know her names.")

In [12]:
app = Flask(__name__)
run_with_ngrok(app)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

model_class = keras.models.load_model("/content/drive/MyDrive/NLP-Project/classmodel_newfull2")
model_gpt2.load_state_dict(torch.load('/content/drive/MyDrive/NLP-Project/model_state_ner_sent2.pt',map_location=torch.device('cpu')))

<All keys matched successfully>

In [13]:
def infer(inp):
    inp = "<startofstring> "+inp+" <reply>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model_gpt2.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output

def getModel(query):
    coreidnp = model_class.predict([query])
    # print(coreidnp)
    coreidnp = np.where(coreidnp > 0.25, 1, 0)
    coreid = coreidnp[0][0]
    if coreid == 0:
        mod = 'gpt2'
    else:
        mod = 'roberta'
    # print("model: ", mod)
    return mod

@app.route("/")
# @cross_origin()
def home():
    return render_template('index.html')

def get_reply(query, last_line):
  try:
    query = query
    reply_gpt = infer(query)
    reply_roberta = ret_roberta(query, reply_gpt, last_line)

    print("Reply:", reply_roberta)
    if profanity.contains_profanity(query):
        return profanity.censor(reply_roberta)
    if profanity.contains_profanity(query):
        return "Please reframe your question with appropriate language!"
    if reply_roberta == "":
        return "Sorry I did not understand!"
    
    return reply_roberta
    
  except Exception as e:
    print(e)
    return "Sorry I did not understand!"

@app.route('/inputQuery', methods=['POST'])
@cross_origin()
def getInput():
    try:
        last_line = ""
        data = request.get_json()
        filename = "/content/drive/MyDrive/NLP-Project/Queries.txt"
        if os.path.getsize(filename) > 0:
          with open(filename, 'r') as f:
            lines = f.readlines()
            last_line = lines[-1].strip()

        reply = get_reply(data["query"],last_line)
        topic_list = data["list_of_topics"]
        # print(topic_list)
        with open("/content/drive/MyDrive/NLP-Project/Queries.txt", "a") as file:
          file.write("Query: "+ data["query"])
          file.write("Reply: "+ reply +"\n")
        return reply
    except Exception as e:
        print(e)
        return "Sorry I did not understand!"


In [146]:
# getModel("covid vaccine")

In [14]:
infer("who is donal trump ? President Trump")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'<startofstring> who is donal trump? President Trump <reply>: they have lots of good young adults that are coming'

In [15]:
get_reply("who is donald trump ?","")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT reply:  they'll all have terrible con stories <endofstring> <ner>: <sentiment>: neutral <pad>
context:  who is donald trump ?   they'll all have terrible con stories <endofstring> <ner>: <sentiment>: neutral <pad> Maybe I can ask Donald Trump for permission. Trump: An American Dream. thats in the US ya but hes a really stickler. I'll have to look it up something else I ran into recently was about the hole in the ozone layer. do you remember that thing?. What thats crazy how could he change the constitution But I mean if any president would do that it would be President Donald Trump He is kinda crazy to be honest. Yeah I agree he has done some really crazy things and cna be kinda out of control sometimes
Reply: President Donald Trump


'President Donald Trump'

In [16]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5fa8-34-168-210-222.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [10/May/2023 14:32:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/May/2023 14:32:42] "GET /static/styles/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/May/2023 14:32:42] "GET /favicon.ico HTTP/1.1" 404 -
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


GPT reply:  Hello! How are you? <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
context:  hello how's it going?.  Hello! How are you? <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> Hello. Hello. Hey there!. Hey, how's it going?. Hello. Hey there!


...

Reply: CleverbotWho


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


GPT reply:  they'll all have terrible con stories <endofstring> <ner>: <sentiment>: neutral <pad>
context:  who is donald trump ? CleverbotWho  they'll all have terrible con stories <endofstring> <ner>: <sentiment>: neutral <pad> Maybe I can ask Donald Trump for permission. Trump: An American Dream. thats in the US ya but hes a really stickler. I'll have to look it up something else I ran into recently was about the hole in the ozone layer. do you remember that thing?. What thats crazy how could he change the constitution But I mean if any president would do that it would be President Donald Trump He is kinda crazy to be honest. Yeah I agree he has done some really crazy things and cna be kinda out of control sometimes


INFO:werkzeug:127.0.0.1 - - [10/May/2023 17:14:18] "POST /inputQuery HTTP/1.1" 200 -


Reply: President Donald Trump
